In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder,MinMaxScaler,OrdinalEncoder

from sklearn.model_selection import train_test_split


import folium
import plotly.express as px

from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb
from sklearn.linear_model import LinearRegression

import time

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA

import scipy as sp

import os


In [2]:
PATH_FOLDER = os.path.join(os.getcwd(), "")

In [3]:
df=pd.read_csv("feature_eng_data.csv")

In [4]:
df.set_index('Index', inplace=True)

In [5]:
df['pk_partition'] = pd.to_datetime(df['pk_partition'])

In [6]:
df

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,...,em_acount,country_id,region_code,gender,age,deceased,salary,Financiacion,Ahorro,Cuentas
Index,,,,,,,,,,,,,,,,,,,,,
0,1375586,2018-01-28,2018-01-12,KHL,1.0,2,0,0,0,0,...,1,0,29.0,0,35,0,87218.10,0,0,1
1,1050611,2018-01-28,2015-08-10,KHE,0.0,3,0,0,0,0,...,1,0,13.0,1,23,0,35548.74,0,0,1
2,1050612,2018-01-28,2015-08-10,KHE,0.0,3,0,0,0,0,...,1,0,13.0,1,23,0,122179.11,0,0,1
3,1050613,2018-01-28,2015-08-10,KHD,0.0,3,1,0,0,0,...,0,0,50.0,0,22,0,119775.54,0,1,0
4,1050614,2018-01-28,2015-08-10,KHE,1.0,3,0,0,0,0,...,1,0,50.0,1,23,0,96796.23,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5962919,1166765,2019-05-28,2016-08-14,KHE,0.0,3,0,0,0,0,...,1,0,50.0,1,22,0,43912.17,0,0,1
5962920,1166764,2019-05-28,2016-08-14,KHE,0.0,3,0,0,0,0,...,1,0,26.0,1,23,0,23334.99,0,0,1
5962921,1166763,2019-05-28,2016-08-14,KHE,1.0,2,0,0,0,0,...,1,0,50.0,0,47,0,96796.23,0,0,1


In [7]:
df['diff_productos'] = df.groupby('pk_cid')['Financiacion'].diff()

In [8]:
df['diff_productos']

Index
0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
          ... 
5962919    0.0
5962920    0.0
5962921    0.0
5962922    0.0
5962923    NaN
Name: diff_productos, Length: 5936633, dtype: float64

In [9]:
df['compra'] = (df['diff_productos'] >= 1).astype(int)

In [10]:
df['compra'].value_counts()

compra
0    5853268
1      83365
Name: count, dtype: int64

In [11]:
#Comprobamos con un pk_cid que sabemso que tiene mas de un producto y vemos que detecta las compras correctamente

df[df['pk_cid'] == 16502]

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,...,region_code,gender,age,deceased,salary,Financiacion,Ahorro,Cuentas,diff_productos,compra
Index,,,,,,,,,,,,,,,,,,,,,
2171490,16502,2018-09-28,2018-09-30,KHE,0.0,3,0,0,0,0,...,28.0,0,57,0,126765.57,0,0,0,NaN,0
2943633,16502,2018-10-28,2018-09-30,KHN,1.0,2,0,0,0,0,...,28.0,0,58,0,126765.57,0,0,1,0.0,0
2962972,16502,2018-11-28,2018-09-30,KHN,1.0,2,0,0,0,0,...,28.0,0,58,0,126765.57,0,0,1,0.0,0
3628237,16502,2018-12-28,2018-09-30,KHN,1.0,2,0,0,0,0,...,28.0,0,58,0,126765.57,0,0,1,0.0,0
4028330,16502,2019-01-28,2018-09-30,KHN,1.0,2,0,0,0,0,...,28.0,0,58,0,126765.57,0,0,2,0.0,0
4480638,16502,2019-02-28,2018-09-30,KHN,1.0,2,0,0,0,0,...,28.0,0,58,0,126765.57,0,0,2,0.0,0
4876038,16502,2019-03-28,2018-09-30,KHN,1.0,2,0,0,0,0,...,28.0,0,58,0,126765.57,0,0,2,0.0,0
5134320,16502,2019-04-28,2018-09-30,KHN,1.0,2,0,0,0,0,...,28.0,0,58,0,126765.57,0,0,2,0.0,0
5523395,16502,2019-05-28,2018-09-30,KHN,1.0,2,0,0,0,0,...,28.0,0,58,0,126765.57,0,0,2,0.0,0


In [12]:
#Llenamos los NaN del primer mes con '0' ya que no tenemos con que comparar 

In [13]:
df['diff_productos'].fillna(value=0, inplace=True)


In [14]:
#Por tanto 'compra' sera nuestro target para la predisposicion de compra de los productos "CUENTAS"

In [15]:
df['entry_date'] = pd.to_datetime(df['entry_date'])
df['pk_partition'] = pd.to_datetime(df['pk_partition'])

In [16]:
df['Antiguedad'] = (df['pk_partition'] - df['entry_date']) / pd.Timedelta(days=30)


In [17]:
df['Antiguedad'] = df['Antiguedad'].round().astype(int)

### TENDECIA DE COMPRA MENSUAL

In [18]:
df_st = df.copy()

In [19]:
df_st['mes'] = df_st['pk_partition'].dt.to_period('M')


In [20]:
serie_temporal = df_st.groupby('mes')['compra'].sum()


In [21]:
serie_temporal

mes
2018-01       0
2018-02    4070
2018-03    4569
2018-04    3990
2018-05    3662
2018-06    4261
2018-07    4548
2018-08    4266
2018-09    5908
2018-10    7001
2018-11    5820
2018-12    5734
2019-01    5391
2019-02    6736
2019-03    6150
2019-04    5529
2019-05    5730
Freq: M, Name: compra, dtype: int32

In [22]:
serie_temporal = serie_temporal.drop(serie_temporal.index[0])

In [23]:
#Intentaremos predecir cuantos productos de CUENTAS venderemos el mes siguiente (JUNIO DE 2019)

df_regresion = serie_temporal.reset_index()
df_regresion['mes'] = df_regresion['mes'].astype(str)  # Convertimos el periodo a string para la regresión lineal

# Dividimos los datos en características (X) y la variable objetivo (y)
X_st = pd.to_numeric(df_regresion['mes'].str.replace('-', ''), errors='coerce').values.reshape(-1, 1)  # Convertimos el periodo a un formato numérico para la regresión lineal
y_st = df_regresion['compra'].values

# Entrenamos el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_st, y_st)

# Predecimos las compras del mes 06 del año 2019
mes_06_2019 = pd.to_numeric('2019-06'.replace('-', ''), errors='coerce').reshape(-1, 1)  # Convertimos el periodo a un formato numérico para la predicción
prediccion_mes_06_2019 = modelo.predict(mes_06_2019)

print("Predicción de compras para el mes 06 del año 2019:", prediccion_mes_06_2019[0])

Predicción de compras para el mes 06 del año 2019: 5993.24768867949


In [24]:
#Predicción de beneficios

Beneficios = prediccion_mes_06_2019[0]*60
Beneficios

#Beneficios estimados para JUNIO 2019 de Productos de FINANCIACION = 359.594 € 

359594.8613207694

### Predicción de acierto del modelo

In [25]:
df

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,...,gender,age,deceased,salary,Financiacion,Ahorro,Cuentas,diff_productos,compra,Antiguedad
Index,,,,,,,,,,,,,,,,,,,,,
0,1375586,2018-01-28,2018-01-12,KHL,1.0,2,0,0,0,0,...,0,35,0,87218.10,0,0,1,0.0,0,1
1,1050611,2018-01-28,2015-08-10,KHE,0.0,3,0,0,0,0,...,1,23,0,35548.74,0,0,1,0.0,0,30
2,1050612,2018-01-28,2015-08-10,KHE,0.0,3,0,0,0,0,...,1,23,0,122179.11,0,0,1,0.0,0,30
3,1050613,2018-01-28,2015-08-10,KHD,0.0,3,1,0,0,0,...,0,22,0,119775.54,0,1,0,0.0,0,30
4,1050614,2018-01-28,2015-08-10,KHE,1.0,3,0,0,0,0,...,1,23,0,96796.23,0,0,1,0.0,0,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5962919,1166765,2019-05-28,2016-08-14,KHE,0.0,3,0,0,0,0,...,1,22,0,43912.17,0,0,1,0.0,0,34
5962920,1166764,2019-05-28,2016-08-14,KHE,0.0,3,0,0,0,0,...,1,23,0,23334.99,0,0,1,0.0,0,34
5962921,1166763,2019-05-28,2016-08-14,KHE,1.0,2,0,0,0,0,...,0,47,0,96796.23,0,0,1,0.0,0,34


In [26]:
df = df.drop(['entry_date','deceased','diff_productos'], axis=1)


In [27]:
df= pd.get_dummies(df, columns=['entry_channel'], drop_first=True)

In [28]:
#Generamos una copia para despues en el recomendador

df_rc2 = df.copy()

In [29]:
df = df.drop(['pk_cid','pk_partition'], axis = 1)

In [30]:
X = df.drop('compra', axis=1)  # Features
y = df['compra']  # Target

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [32]:
print("Tamaño del conjunto de entrenamiento (X):", X_train.shape)
print("Tamaño del conjunto de prueba (X):", X_test.shape)
print("Tamaño del conjunto de entrenamiento (y):", y_train.shape)
print("Tamaño del conjunto de prueba (y):", y_test.shape)

Tamaño del conjunto de entrenamiento (X): (4749306, 38)
Tamaño del conjunto de prueba (X): (1187327, 38)
Tamaño del conjunto de entrenamiento (y): (4749306,)
Tamaño del conjunto de prueba (y): (1187327,)


In [33]:
def AUC(model, X, y):
    return roc_auc_score(y, model.predict_proba(X)[:, 1])

def cross_validation(model, X, y):
    Kfold_metric = cross_val_score(model, X, y, cv=4, scoring='roc_auc')
    return Kfold_metric.mean()

In [34]:
label_ = []
metric_train_ = []
metric_valida_ = []

In [35]:
def evaluate_model(label, model, X, y):
 model.fit(X,y)
 metric_train = AUC(model,X,y)
 metric_valida = cross_validation(model,X,y)
 
 label_.append(label)
 metric_train_.append(metric_train)
 metric_valida_.append(metric_valida)
 return pd.DataFrame({'Model': label_,
                'AUC Train': metric_train_,
                'AUC Cross-Valida': metric_valida_
                }).sort_values('AUC Cross-Valida')

evaluate_model(label='XGBoost', 
 model = xgb.XGBClassifier(), 
 X=X_train, y=y_train)

### Recomendador

In [36]:
df_rc=pd.read_csv("clusters_v2.csv",index_col=0)

In [37]:
df_rc2 = df_rc2[df_rc2["pk_partition"] == "2019-05-28"]

In [38]:
df_rc2

,pk_cid,pk_partition,active_customer,segment,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,...,entry_channel_KHD,entry_channel_KHE,entry_channel_KHF,entry_channel_KHK,entry_channel_KHL,entry_channel_KHM,entry_channel_KHN,entry_channel_KHO,entry_channel_KHQ,entry_channel_RED
Index,,,,,,,,,,,,,,,,,,,,,
5519930,657817,2019-05-28,0.0,3,0,0,0,0,0,0,...,False,True,False,False,False,False,False,False,False,False
5519932,657905,2019-05-28,1.0,1,0,0,0,0,0,1,...,False,False,False,False,False,False,False,False,False,False
5519933,657336,2019-05-28,1.0,2,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
5519934,657147,2019-05-28,0.0,2,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
5519935,658184,2019-05-28,1.0,2,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5962919,1166765,2019-05-28,0.0,3,0,0,0,0,0,0,...,False,True,False,False,False,False,False,False,False,False
5962920,1166764,2019-05-28,0.0,3,0,0,0,0,0,0,...,False,True,False,False,False,False,False,False,False,False
5962921,1166763,2019-05-28,1.0,2,0,0,0,0,0,0,...,False,True,False,False,False,False,False,False,False,False


In [39]:
df_rc2 = df_rc2[["pk_cid","salary","age","segment","gender","region_code", "Financiacion","Ahorro","Cuentas","Antiguedad","compra"]]

In [40]:
df_rc3 = df_rc2.merge(df_rc["cluster"], left_on="pk_cid", right_index=True, how="inner")


In [41]:
df_rc3 = df_rc3.set_index("pk_cid")

In [42]:
df_cluster_4 = df_rc3[df_rc3["cluster"] == 4].copy()

In [43]:
df_cluster_4

,salary,age,segment,gender,region_code,Financiacion,Ahorro,Cuentas,Antiguedad,compra,cluster
pk_cid,,,,,,,,,,,
658184,183036.72,42,2,1,28.0,2,1,3,49,0,4
656415,143418.03,32,2,1,28.0,2,1,2,52,1,4
650478,90712.98,86,2,0,28.0,1,1,2,9,0,4
654801,67164.33,63,1,0,28.0,0,2,1,28,0,4
652139,34699.20,85,2,0,35.0,1,1,1,42,0,4
...,...,...,...,...,...,...,...,...,...,...,...
1167084,180860.97,33,2,1,28.0,1,0,1,34,0,4
1167081,52423.38,35,2,0,28.0,1,0,2,34,0,4
1166671,148876.17,34,2,0,46.0,1,1,2,34,0,4


In [44]:
# eliminamos la columna de cluster del dataset para que no se use en el cálculo de la similitud de coseno.
df_cluster_4.drop("cluster", axis = 1, inplace = True)

In [45]:
df_cluster_4 = df_cluster_4.sample(frac=0.1, random_state=42)

In [46]:
#Es el usuario que usaremos como base para la recomendacion, teniendo en cuenta que haya hecho al menos una compra en el ultimo mes 

usuario_estrella = df_cluster_4.loc[df_cluster_4['compra'] == 1].index[0]

In [47]:
EXECUTE = True

In [48]:
if EXECUTE:
    # normalizamos nuestro dataset
    st = time.time()
    
    scaler = MinMaxScaler()
    df_normalized = scaler.fit_transform(df_cluster_4)
    df_normalized = pd.DataFrame(df_normalized, columns=df_cluster_4.columns)
    df_normalized.index = df_cluster_4.index


    et = time.time()
    print("Normalization took {} minutes".format(round((et - st)/60, 2)))

    sparse_ratings = sp.sparse.csr_matrix(df_normalized.values)

    st = time.time()
    print("Working with user similarity")
    user_similarity = cosine_similarity(sparse_ratings)
    user_sim_df = pd.DataFrame(user_similarity, index = df_normalized.index, columns = df_normalized.index)
    user_sim_df.columns = map(str, user_sim_df.columns)
    user_sim_df.to_parquet("cf_user_similarity.parquet.gzip")

    print("Working with item similarity")
    item_similarity = cosine_similarity(sparse_ratings.T)
    item_sim_df = pd.DataFrame(item_similarity, index = df_normalized.columns, columns = df_normalized.columns)
    item_sim_df.columns = map(str, item_sim_df.columns)
    item_sim_df.to_parquet("cf_item_similarity.parquet.gzip")

    et = time.time()
    print("Total time to calculate similarity took {} minutes.".format(round((et - st)/60, 2)))

else:
    user_sim_df = pd.read_parquet(os.path.join(PATH_FOLDER, "cf_user_similarity.parquet.gzip"))
    item_sim_df = pd.read_parquet(os.path.join(PATH_FOLDER, "cf_item_similarity.parquet.gzip"))

Normalization took 0.0 minutes
Working with user similarity
Working with item similarity
Total time to calculate similarity took 0.01 minutes.


In [49]:
user_sim_df.head(5)

,1382437,1376606,1113048,1056596,1092469,1096408,1297357,1262644,1121584,1023660,...,1120382,1021918,1052967,1401942,1136250,1365630,1105940,1329109,1263015,1167394
pk_cid,,,,,,,,,,,,,,,,,,,,,
1382437,1.000000,0.944461,0.710006,0.655465,0.813611,0.718715,0.497904,0.542052,0.622849,0.688163,...,0.649851,0.770090,0.708889,0.557839,0.909339,0.521430,0.718587,0.872373,0.647050,0.886212
1376606,0.944461,1.000000,0.715940,0.708289,0.904807,0.655183,0.617418,0.637217,0.716379,0.681743,...,0.610096,0.689065,0.702550,0.611283,0.927880,0.554039,0.663411,0.967227,0.651174,0.951365
1113048,0.710006,0.715940,1.000000,0.960642,0.750967,0.877642,0.899089,0.915767,0.796329,0.869565,...,0.891179,0.723448,0.906731,0.922227,0.742067,0.781358,0.886355,0.725042,0.895007,0.751780
1056596,0.655465,0.708289,0.960642,1.000000,0.776899,0.882077,0.927037,0.949162,0.914158,0.838989,...,0.899286,0.702670,0.943330,0.927898,0.748196,0.871292,0.890914,0.728989,0.912173,0.767414
1092469,0.813611,0.904807,0.750967,0.776899,1.000000,0.657399,0.670621,0.710579,0.765725,0.717441,...,0.637960,0.764568,0.771027,0.600452,0.947058,0.498329,0.669122,0.962522,0.681868,0.982557


In [50]:
item_sim_df.head(5)

,salary,age,segment,gender,region_code,Financiacion,Ahorro,Cuentas,Antiguedad,compra
salary,1.000000,0.772053,0.791849,0.643954,0.790228,0.758121,0.735650,0.831872,0.810541,0.231276
age,0.772053,1.000000,0.693201,0.647381,0.787716,0.717613,0.715467,0.778718,0.795987,0.173341
segment,0.791849,0.693201,1.000000,0.667329,0.821748,0.794786,0.712788,0.854508,0.834250,0.212527
gender,0.643954,0.647381,0.667329,1.000000,0.681189,0.647564,0.598680,0.698294,0.678562,0.167310
region_code,0.790228,0.787716,0.821748,0.681189,1.000000,0.762718,0.758758,0.852031,0.822006,0.189115


In [51]:
def top_users(user, df):
    '''
    This function prints the top 790 similar users based on cosine similarity.
    '''

    df.columns = map(int, df.columns)

    if user not in df.columns:
        return('No data available on user {}'.format(user))

    print('Most Similar Users:\n')

    sim_users = df.sort_values(by = user, ascending=False).index[1:789]
    sim_values = df.sort_values(by = user, ascending=False).loc[:,user].tolist()[1:789]

    for user, sim in zip(sim_users, sim_values):
        print('User #{0}, Similarity value: {1:.4f}'.format(user, sim))

    return sim_users, sim_values

In [52]:
[similar_users, similar_values] = top_users(usuario_estrella, user_sim_df)

Most Similar Users:

User #891259, Similarity value: 0.9888
User #1260764, Similarity value: 0.9677
User #1136823, Similarity value: 0.9676
User #1107962, Similarity value: 0.9655
User #1025017, Similarity value: 0.9620
User #1008654, Similarity value: 0.9609
User #1133563, Similarity value: 0.9551
User #1002324, Similarity value: 0.9477
User #1042758, Similarity value: 0.9393
User #1121038, Similarity value: 0.9382
User #1135051, Similarity value: 0.9373
User #1266960, Similarity value: 0.9360
User #1216926, Similarity value: 0.9349
User #1333272, Similarity value: 0.9321
User #1053696, Similarity value: 0.9296
User #1403165, Similarity value: 0.9286
User #1093489, Similarity value: 0.9263
User #1357204, Similarity value: 0.9257
User #1369871, Similarity value: 0.9116
User #1312344, Similarity value: 0.9073
User #1203146, Similarity value: 0.9042
User #1009394, Similarity value: 0.9040
User #1375393, Similarity value: 0.9015
User #1489691, Similarity value: 0.8974
User #1297221, Simil

In [53]:
similar_values

[0.9888468440947606,
 0.9676904343794188,
 0.9676389136941338,
 0.9655433609080252,
 0.9619830957533272,
 0.9609305464055011,
 0.9550745057447947,
 0.947687868577914,
 0.9393411959072062,
 0.9382271035584964,
 0.9372924963948335,
 0.9360453366079328,
 0.9349465607609858,
 0.9320716241353817,
 0.9296339002893458,
 0.9286251388517325,
 0.9263403809998283,
 0.9256613037801696,
 0.9116281722472759,
 0.9072595865460131,
 0.9042315446306836,
 0.9040190148026654,
 0.9015120823088874,
 0.8973566679244163,
 0.8950903943857327,
 0.8921042988097942,
 0.8901932952806503,
 0.8850014935198951,
 0.8843335939417813,
 0.8834763175423407,
 0.8834646811450706,
 0.883453389794464,
 0.883173351924492,
 0.8828898229211455,
 0.8820352320750429,
 0.8805856140049206,
 0.879560403035011,
 0.8788101543510274,
 0.8787495253830272,
 0.8772130075466495,
 0.8771950533177174,
 0.8755210707337133,
 0.8750704145784359,
 0.8739361247743386,
 0.8730628940121287,
 0.8720417639322084,
 0.8720098531390768,
 0.87138994177210

In [54]:
media = np.mean(similar_values[:789])
media

0.8051873235453209